# Import Libraries & methods

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Flatten, Input, BatchNormalization, Dense, Activation, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.applications.vgg19 import preprocess_input

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder

from copy import deepcopy
from datetime import datetime

### Methods used in FedProx Algorithm

In [ ]:

def loss_classifier(predictions, labels):
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=predictions)
    return tf.reduce_mean(loss)


def loss_dataset(model, dataset, loss_f):
    loss = 0
    for idx, (features, labels) in enumerate(dataset):
        # Convert PyTorch tensors to NumPy and then to TensorFlow tensors
        features_np = features.detach().cpu().numpy()
        labels_np = labels.detach().cpu().numpy()

        features_tf = tf.convert_to_tensor(features_np, dtype=tf.float32)
        labels_tf = tf.convert_to_tensor(labels_np, dtype=tf.int32)

        predictions = model(features_tf)
        loss += loss_f(predictions, labels_tf)
    loss /= (idx + 1)
    return loss


def accuracy_dataset(model, dataset):
    correct = 0
    total = 0
    for features, labels in dataset:
        # Convert PyTorch tensors to NumPy and then to TensorFlow tensors
        features_np = features.detach().cpu().numpy()
        labels_np = labels.detach().cpu().numpy()

        features_tf = tf.convert_to_tensor(features_np, dtype=tf.float32)
        labels_tf = tf.convert_to_tensor(labels_np, dtype=tf.int32)

        predictions = model(features_tf)
        predicted = tf.argmax(predictions, axis=1, output_type=tf.int32)
        correct += tf.reduce_sum(tf.cast(tf.equal(predicted, labels_tf), tf.int32)).numpy()
        total += labels_tf.shape[0]

    accuracy = 100 * correct / total
    return accuracy


def train_step(model, model_0, mu, optimizer, train_data, loss_f):
    total_loss = 0
    for idx, (features, labels) in enumerate(train_data):
        # Convert PyTorch tensors to NumPy and then to TensorFlow tensors
        features_np = features.detach().cpu().numpy()
        labels_np = labels.detach().cpu().numpy()

        features_tf = tf.convert_to_tensor(features_np, dtype=tf.float32)
        labels_tf = tf.convert_to_tensor(labels_np, dtype=tf.int32)

        with tf.GradientTape() as tape:
            predictions = model(features_tf)
            loss = loss_f(predictions, labels_tf)
            loss += mu / 2 * difference_models_norm_2(model, model_0)

        total_loss += loss
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return total_loss / (idx + 1)


def local_learning(model, mu, optimizer, train_data, epochs, loss_f):
    # (No longer used by FedProx, but left intact)
    model_0 = tf.keras.models.clone_model(model)
    model_0.set_weights(model.get_weights())

    for e in range(epochs):
        local_loss = train_step(model, model_0, mu, optimizer, train_data, loss_f)

    return local_loss


def difference_models_norm_2(model_1, model_2):
    norm = tf.reduce_sum([tf.reduce_sum(tf.square(w1 - w2))
                          for w1, w2 in zip(model_1.trainable_variables, model_2.trainable_variables)])
    return norm


def set_to_zero_model_weights(model):
    for layer_weights in model.trainable_variables:
        layer_weights.assign(tf.zeros_like(layer_weights))


def average_models(model, clients_models_hist, weights):
    set_to_zero_model_weights(model)
    for k, client_hist in enumerate(clients_models_hist):
        for idx, layer_weights in enumerate(model.trainable_variables):
            contribution = client_hist[idx] * weights[k]
            layer_weights.assign_add(contribution)



### Federated Proximal Algorithm

In [ ]:
def FedProx(model, training_sets, n_iter, testing_sets, mu=0.1, epochs=1, lr=0.0001, decay=1):
    # Verify that `model` is a Keras model instance
    if not isinstance(model, tf.keras.Model):
        raise TypeError("The provided model is not a TensorFlow Keras model. Please provide a valid Keras model.")

    loss_f = loss_classifier
    K = len(training_sets)
    n_samples = sum([len(db) for db in training_sets])
    weights = [len(db) / n_samples for db in training_sets]
    print("Clients' weights:", weights)

    # Initialize history lists for training and testing
    train_loss_hist = []
    train_acc_hist = []
    test_loss_hist = []
    test_acc_hist = []
    models_hist = []

    for i in range(n_iter):
        clients_params = []
        clients_losses = []
        clients_accuracies = []

        for k in range(K):
            # Clone the global model for local training
            local_model = tf.keras.models.clone_model(model)
            local_model.set_weights(model.get_weights())

            # ---- Two‐Phase Local Update ----
            # Prepare a copy of initial weights for FedProx proximal term
            model_0 = tf.keras.models.clone_model(local_model)
            model_0.set_weights(local_model.get_weights())

            # # Phase 1: freeze backbone, train 5 epochs at lr=1e-3
            local_model.layers[1].trainable = False
            optimizer1 = tf.keras.optimizers.Adam(1e-3)
            for _ in range(1):
                local_loss = train_step(local_model, model_0, mu, optimizer1, training_sets[k], loss_f)

            # Phase 2: unfreeze backbone, train `epochs` epochs at lr=1e-4
            local_model.layers[1].trainable = True
            optimizer2 = tf.keras.optimizers.SGD(learning_rate=1e-4, momentum=0.9)
            for _ in range(epochs):
                local_loss = train_step(local_model, model_0, mu, optimizer2, training_sets[k], loss_f)

            clients_losses.append(local_loss)

            # Track training accuracy for the client
            train_acc = accuracy_dataset(local_model, training_sets[k])
            clients_accuracies.append(train_acc)

            # Store model parameters
            clients_params.append([tf.identity(tens_param) for tens_param in local_model.trainable_variables])
        # ------------------------------

        # Aggregate back to the global model
        average_models(model, clients_params, weights=weights)
        models_hist.append(deepcopy(clients_params))

        # Record metrics
        train_loss_hist.append(clients_losses)
        train_acc_hist.append(clients_accuracies)

        test_loss_hist.append([loss_dataset(model, dl, loss_f).numpy() for dl in testing_sets])
        test_acc_hist.append([accuracy_dataset(model, dl) for dl in testing_sets])

        lr *= decay
        print(f'====> Round {i+1} Server Test Accuracy: {test_acc_hist[-1]}')

    return model, train_loss_hist, train_acc_hist, test_loss_hist, test_acc_hist




# To save model metrics

In [ ]:
# Save the history to a file with detailed formatting
def save_history_to_file(filename, train_loss, train_acc, test_loss, test_acc):
    with open(filename, "w") as f:
        f.write("FedProx Training and Testing Metrics\n")
        f.write("=" * 50 + "\n")
        
        for i in range(len(train_loss)):
            try:
                train_loss_str = [float(tensor.numpy()) if hasattr(tensor, 'numpy') else float(tensor) 
                                  for tensor in train_loss[i]]
                train_acc_str  = [float(val) for val in train_acc[i]]
                test_loss_str  = [float(val) for val in test_loss[i]]
                test_acc_str   = [float(val) for val in test_acc[i]]
                
                f.write(f"Iteration {i + 1}:\n")
                f.write(f"Train Loss: {train_loss_str}\n")
                f.write(f"Train Accuracy: {train_acc_str}\n")
                f.write(f"Test Loss: {test_loss_str}\n")
                f.write(f"Test Accuracy: {test_acc_str}\n")
                f.write("-" * 50 + "\n")
                
            except (ValueError, TypeError) as e:
                f.write(f"Error processing round {i + 1}: {e}\n")
                f.write("-" * 50 + "\n")
    
    print(f"History saved to {filename}")



# Load training and testing data

In [ ]:
# Directories for each client
split_dirs = [
    "NON_IID/Model_1",
    "NON_IID/Model_2",
    "NON_IID/Model_3",
    "NON_IID/Model_4"
]

# Number of clients
n_clients = len(split_dirs)

# Updated image transformations: Normalize first, then permute
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),          
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225]),
    transforms.Lambda(lambda x: x.permute(1, 2, 0))
])

def get_tumor_dataloaders(split_dirs, batch_size=25, shuffle=True):
    tumor_iid_train_dls = []
    tumor_iid_test_dls = []

    for client_idx, client_dir in enumerate(split_dirs):
        train_dir = os.path.join(client_dir, 'train')
        test_dir  = os.path.join(client_dir, 'test')

        if not os.path.exists(train_dir) or not os.path.exists(test_dir):
            print(f"Directory not found for client {client_idx + 1}: {train_dir}, {test_dir}")
            continue

        train_dataset = datasets.ImageFolder(train_dir, transform=transform)
        train_loader  = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
        test_dataset  = datasets.ImageFolder(test_dir,  transform=transform)
        test_loader   = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)

        tumor_iid_train_dls.append(train_loader)
        tumor_iid_test_dls.append(test_loader)

    return tumor_iid_train_dls, tumor_iid_test_dls

Tumor_iid_train_dls, Tumor_iid_test_dls = get_tumor_dataloaders(split_dirs, batch_size=25)

for batch_idx, (images, labels) in enumerate(Tumor_iid_train_dls[0]):
    print(f"Batch {batch_idx} image sizes: {images.size()}")
    break


# ResNet50 model 

In [ ]:
# ---------------- Model Definition ------------------
def ResNet50_Model():
    input_shape = (224, 224, 3)
    base_model = tf.keras.applications.ResNet50(
        include_top=False,
        input_shape=input_shape,
        weights='imagenet'
    )
 
    x = layers.GlobalAveragePooling2D()(x)

    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dense(512,  activation='relu')(x)
    x = layers.Dense(256,  activation='relu')(x)
    x = layers.Dense(128,  activation='relu')(x)
    x = layers.Dropout(0.5)(x)

    # Optional dense attention head
    x = layers.Reshape((1, 1, 128))(x)
    x_max = layers.GlobalMaxPooling2D()(x)
    x_avg = layers.GlobalAveragePooling2D()(x)
    x = layers.Concatenate()([x_max, x_avg])

    outputs = layers.Dense(4, activation='softmax')(x)
    return models.Model(inputs=base_model.input, outputs=outputs, name='ResNet50_Model')



# Model Intialization

In [ ]:
# Instantiate & compile the global model
model = ResNet50_Model()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Load original testing dataset (without augmentation)

In [ ]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),          # Convert image to tensor format (C, H, W)
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
    transforms.Lambda(lambda x: x.permute(1, 2, 0))  # Permute to (H, W, C)
])

# Load the testing dataset
test_dir = 'BrainTumor_MRI/Testing'
test_dataset = datasets.ImageFolder(test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Class names mapping (adjust if necessary)
class_names = test_dataset.classes

# To evaluate the trained model
# Save the confusion matrix + performance metrics

In [ ]:
# Define class names
class_names = ["glioma", "meningioma", "notumor", "pituitary"]  # Modify according to your classes
MODEL_PATH = 'model_results_mu=1.0/'

# Function to evaluate the global model on the testing dataset
def evaluate_global_model(model, test_loader, output_file=f'{MODEL_PATH}Global_Model_Results_mu=1.0.txt'):
    all_preds = []
    all_labels = []
    total_loss = 0

    for features, labels in test_loader:
        # Convert to TensorFlow tensors
        features_np = features.detach().cpu().numpy()
        labels_np = labels.detach().cpu().numpy()

        features_tf = tf.convert_to_tensor(features_np, dtype=tf.float32)
        labels_tf = tf.convert_to_tensor(labels_np, dtype=tf.int32)

        # Get predictions
        predictions = model(features_tf)
        predicted = tf.argmax(predictions, axis=1, output_type=tf.int32)

        # Accumulate predictions and labels
        all_preds.extend(predicted.numpy())
        all_labels.extend(labels_np)

        # Compute loss
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels_tf, logits=predictions)
        total_loss += tf.reduce_mean(loss).numpy()

    # Compute metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    conf_matrix = confusion_matrix(all_labels, all_preds)

    # Classification report
    classification_report_str = classification_report(all_labels, all_preds, target_names=class_names)

    # Append metrics to a file
    with open(output_file, "a") as file:
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')  # Get current timestamp
        file.write(f"Evaluation Timestamp: {timestamp}\n")
        file.write("Evaluation Metrics:\n")
        file.write(f"Accuracy: {accuracy * 100:.2f}%\n")
        file.write(f"Loss: {total_loss / len(test_loader):.4f}\n")
        file.write(f"Precision: {precision:.4f}\n")
        file.write(f"Recall: {recall:.4f}\n")
        file.write(f"F1 Score: {f1:.4f}\n\n")
        file.write("Classification Report:\n")
        file.write(classification_report_str)
        file.write("\nConfusion Matrix:\n")
        file.write("\n".join(["\t".join(map(str, row)) for row in conf_matrix]))
        file.write("\n" + "-" * 50 + "\n")

    # Print metrics
    print("\nClassification Report:\n", classification_report_str)
    print(f"\nAccuracy: {accuracy * 100:.2f}%")
    print(f"Loss: {total_loss / len(test_loader):.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    # Plot and save heatmap
    plt.figure(figsize=(10, 7))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix')
    heatmap_path = f'model_results_mu=1.0/confusion_matrix_{timestamp.replace(" ", "_").replace(":", "-")}.png'
    plt.savefig(heatmap_path)  # Save as PNG with timestamp
    plt.close()  # Close the figure
    
    current_loss = (total_loss / len(test_loader))
    return current_loss


# FedProx hyper parameters

In [ ]:
# Initial parameters
MODEL_PATH = 'model_results_mu=1.0/' # to save the model
n_iter = 100
rounds_per_segment = 5 # for every 5 rounds global model will evalaute and save results
epochs = 1

### mu & learning rate

### change the mu value like mu=0 || mu=0.1 || mu=0.4 || mu=0.7 || mu=0.1

In [ ]:
lr = 0.0001
mu = 1.0

In [ ]:
# Initialize lists to store cumulative training and testing metrics
all_train_loss = []
all_train_acc = []
all_test_loss = []
all_test_acc = []

## 100 rounds training

In [ ]:
# Initialize early stopping parameters
patience = 20
best_metric = None  # Track the lowest validation loss
no_improve_rounds = 0  # Counter for rounds without improvement

### Load the model if the training is stoped in the middle beacuse of memory exhaust or any interuption. Then use below code to load the saved model in last round

In [ ]:
# model = tf.keras.models.load_model('model_results_mu=1.0/best_model_rounds_5_mu=1.0.h5')

In [ ]:
# Function for FedProx training in segments
for i in range(0, n_iter, rounds_per_segment):
    print(f"Starting rounds {i+1} to {i+rounds_per_segment}")
    
    # Train for the specified number of rounds in this segment
    model, train_loss_f, train_acc_f, test_loss_f, test_acc_f = FedProx(
        model, Tumor_iid_train_dls, rounds_per_segment, Tumor_iid_test_dls, mu=mu, epochs=epochs, lr=lr
    )
    
    # Append the metrics to the cumulative lists
    all_train_loss.extend(train_loss_f)
    all_train_acc.extend(train_acc_f)
    all_test_loss.extend(test_loss_f)
    all_test_acc.extend(test_acc_f)
    
    # Save the model at the end of each segment
    segment_model_path = f'{MODEL_PATH}global_model_rounds_{i+rounds_per_segment}_mu={mu}.h5'
    model.save(segment_model_path)
    print(f"Model saved at: {segment_model_path}")
    
    # Test the saved global model on the testing dataset
    print(f"Evaluating global model for rounds {i+1} to {i+rounds_per_segment}")
    current_metric = evaluate_global_model(model, test_loader, output_file=f'{MODEL_PATH}Global_Model_Results_mu={mu}.txt')
    
    # Check for early stopping based on validation loss. Use the most recent validation loss
    print(f"Current validation loss: {current_metric:.4f}")
    
    if best_metric is None or current_metric < best_metric:  # Improvement check
        best_metric = current_metric
        no_improve_rounds = 0  # Reset counter
        # Save the best model
        best_model_path = f'{MODEL_PATH}best_model_rounds_{i+rounds_per_segment}_mu={mu}.h5'
        model.save(best_model_path)
        print(f"Best model updated and saved at: {best_model_path}")
    else:
        no_improve_rounds += 1
        print(f"No improvement for {no_improve_rounds} rounds. Best validation loss: {best_metric:.4f}")
    
    if no_improve_rounds >= patience:  # Early stopping condition
        print("Early stopping triggered. Training stopped.")
        break
    
    # Reload the model to ensure continuity for the next segment
    model = tf.keras.models.load_model(segment_model_path)


### If training the model 100 rounds completed in one, save the performance metrics (Optional). Metrics of global model saved for every 5 rounds, during evaluation

In [ ]:
# Save the cumulative history to a file after all rounds are complete
train_loss = all_train_loss[-100:]  # Adjusted to reflect 30 rounds for rounds 51 to 80
train_acc = all_train_acc[-100:]
test_loss = all_test_loss[-100:]
test_acc = all_test_acc[-100:]

# File to save the results
output_file = f'{MODEL_PATH}results-mu={mu}_{len(train_loss)}_rounds.txt'

# Save the data to the file
save_history_to_file(output_file, train_loss, train_acc, test_loss, test_acc)


# Load the trained model to test it

In [ ]:
MODEL_PATH = f'{MODEL_PATH}global_model_rounds_100_mu={mu}.h5'
model = tf.keras.models.load_model(MODEL_PATH)

In [ ]:
evaluate_global_model(model, test_loader)

# to check how good the model to predict the class of a random image from testing data

In [ ]:
# Define class names
class_names = ['glioma', 'meningioma', 'notumor', 'pituitary']

# Function to predict the class of a single image
def predict_image(image_path, model):
    # Load and preprocess the image
    image = load_img(image_path, target_size=(224, 224))  # Resize to 224x224
    image_array = img_to_array(image)  # Convert to numpy array
    image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension
    image_array = preprocess_input(image_array)  # Normalize for VGG19

    # Perform inference
    predictions = model(image_array, training=False)  # Inference mode
    probabilities = tf.nn.softmax(predictions[0]).numpy()  # Apply softmax
    predicted_class = np.argmax(probabilities)

    # Print the predicted class and probabilities
    print(f"Predicted Class: {class_names[predicted_class]}")
    print(f"Probabilities: {probabilities}")


In [ ]:
image_path = 'BrainTumor_MRI/Testing/meningioma/Te-me_0073.jpg'
predict_image(image_path, model)